In [2]:
import boto3
import sagemaker
%matplotlib inline
%config InlineBackend.figure_format='retina'
#from pyathena import connect
import pandas as pd
import numpy as np

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [3]:
ingest_create_athena_db_passed = False

In [4]:
%store -r ingest_create_athena_table_parquet_passed

In [5]:
database_name = 'flight_analysis'
table1_name = 'cancellation_codes_4'

In [6]:
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [9]:
!pip install PyAthena
from pyathena import connect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.4/67.4 kB 1.1 MB/s eta 0:00:00ta 0:00:01


In [10]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [11]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)
pd.read_sql(statement, conn)

CREATE DATABASE IF NOT EXISTS flight_analysis


""


In [12]:
cancellation_s3 = 's3://sagemaker-us-east-1-993410942383/content-project/airline_data/cancellation_table'
airport_codes_s3 = 's3://sagemaker-us-east-1-993410942383/content-project/airline_data/airport_codes_table'
WeatherUS_2016_s3 = 's3://sagemaker-us-east-1-993410942383/content-project/airline_data/WeatherUS_2016'
flights_s3 = 's3://sagemaker-us-east-1-993410942383/content-project/airline_data/airline_flights_table'

In [13]:
table1_name = 'cancellation_codes_4'
table2_name = 'airport_codes_table'
table3_name = 'WeatherUS_2016'
table_name_flights = 'flights'

In [14]:
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         Index string,
         Code string,
         Description string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table1_name, cancellation_s3
)

In [15]:
pd.read_sql(statement, conn)

""


In [16]:
query = "SELECT * FROM {}.{}".format(database_name, table1_name)
pd.read_sql(query, conn)

,index,code,description
0,0,A,Carrier
1,1,B,Weather
2,2,C,National Air System
3,3,D,Security


In [17]:
statement2 = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         Index string,
         Code string,
         Description string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table2_name, airport_codes_s3
)

In [18]:
pd.read_sql(statement2, conn)

""


In [19]:
query2 = "SELECT * FROM {}.{}".format(database_name, table2_name)
pd.read_sql(query2, conn)

,index,code,description
0,0,01A,"""Afognak Lake"
1,1,03A,"""Granite Mountain"
2,2,04A,"""Lik"
3,3,05A,"""Little Squaw"
4,4,06A,"""Kizhuyak"
...,...,...,...
6380,6380,ZXZ,"""Waterville"
6381,6381,ZYL,"""Sylhet"
6382,6382,ZZU,"""Mzuzu"
6383,6383,ZZV,"""Zanesville"


In [20]:
statement3 = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         Index string,
         EventId string,
         Type string,
         Severity string,
         StartTime string,
         EndTime string,
         Precipitation string,
         TimeZone string,
         AirportCode string,
         LocationLat string,
         LocationLng string,
         City string,
         County string,
         State string,
         ZipCode string
        
)
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table3_name, WeatherUS_2016_s3
)

In [21]:
pd.read_sql(statement3, conn)

""


In [22]:
pd.read_sql(f'SELECT * FROM {database_name}.{table3_name} LIMIT 5', conn)

,index,eventid,type,severity,starttime,endtime,precipitation,timezone,airportcode,locationlat,locationlng,city,county,state,zipcode
0,6109752,W-6189592,Fog,Moderate,2016-01-12 19:34:00,2016-01-12 20:34:00,0.0,US/Eastern,KCVX,45.3047,-85.2747,Charlevoix,Charlevoix,MI,49720.0
1,6109753,W-6189593,Fog,Moderate,2016-01-12 20:54:00,2016-01-12 21:14:00,0.0,US/Eastern,KCVX,45.3047,-85.2747,Charlevoix,Charlevoix,MI,49720.0
2,6109754,W-6189594,Fog,Severe,2016-01-12 21:35:00,2016-01-12 21:55:00,0.0,US/Eastern,KCVX,45.3047,-85.2747,Charlevoix,Charlevoix,MI,49720.0
3,6109755,W-6189595,Fog,Moderate,2016-01-12 21:55:00,2016-01-12 22:14:00,0.0,US/Eastern,KCVX,45.3047,-85.2747,Charlevoix,Charlevoix,MI,49720.0
4,6109756,W-6189596,Snow,Light,2016-01-12 22:14:00,2016-01-12 22:35:00,0.0,US/Eastern,KCVX,45.3047,-85.2747,Charlevoix,Charlevoix,MI,49720.0


In [23]:
statement6 = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         Index string,
         Year	int,
         Quarter	int,
         Month 	int,
         DayofMonth 	int,
         DayOfWeek 	int,
         FlightDate 	string,
         UniqueCarrier 	string,
         AirlineID	int,
         Carrier	string,
         TailNum	string,
         FlightNum	int,
         OriginAirportID	int,
         OriginAirportSeqID	int,
         OriginCityMarketID	int,
         Origin	string,
         OriginCityName	string,
         OriginState	string,
         OriginStateFips	int,
         OriginStateName	string,
         OriginWac	int,
         DestAirportID	int,
         DestAirportSeqID	int,
         DestCityMarketID	int,
         Dest	string,
         DestCityName	string,
         DestState	string,
         DestStateFips	int,
         DestStateName	string,
         DestWac	int,
         CRSDepTime	int,
         DepTime	float,
         DepDelay	float,
         DepDelayMinutes	float,
         DepDel15	float,
         DepartureDelayGroups	float,
         DepTimeBlk	string,
         TaxiOut	float,
         WheelsOff	float,
         WheelsOn	float,
         TaxiIn	float,
         CRSArrTime	int,
         ArrTime	float,
         ArrDelay	float,
         ArrDelayMinutes	float,
         ArrDel15	float,
         ArrivalDelayGroups	float,
         ArrTimeBlk	string,
         Cancelled	int,
         CancellationCode	float,
         Diverted	int,
         CRSElapsedTime	float,
         ActualElapsedTime	float,
         AirTime	float,
         Flights	float,
         Distance	float,
         DistanceGroup	int,
         CarrierDelay	float,
         WeatherDelay	float,
         NASDelay	float,
         SecurityDelay	float,
         LateAircraftDelay	float,
         FirstDepTime	float,
         TotalAddGTime	float,
         LongestAddGTime	float,
         DivAirportLandings	int,
         DivReachedDest	float,
         DivActualElapsedTime	float,
         DivArrDelay	float,
         DivDistance	float,
         Div1Airport	string,
         Div1AirportID	float,
         Div1AirportSeqID	float,
         Div1WheelsOn	float,
         Div1TotalGTime	float,
         Div1LongestGTime	float,
         Div1WheelsOff	float,
         Div1TailNum	string,
         Div2Airport	string,
         Div2AirportID	float,
         Div2AirportSeqID	float,
         Div2WheelsOn	float,
         Div2TotalGTime	float,
         Div2LongestGTime	float
        
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
 LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name_flights, flights_s3
)


In [24]:
pd.read_sql(statement6, conn)

""


In [25]:
query_flights = "SELECT Index, FlightDate, FlightNum, OriginAirportID, OriginAirportSeqID, OriginCityMarketID, Origin, OriginCityName, OriginState, OriginStateFips, OriginStateName, Dest FROM {}.{} ORDER BY Index asc LIMIT 10".format(database_name, table_name_flights)
pd.read_sql(query_flights, conn)

,Index,FlightDate,FlightNum,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,Dest
0,0,2016-01-06,43,11298,1129804,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,DTW
1,1,2016-01-07,43,11298,1129804,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,DTW
2,10,2016-01-16,43,11298,1129804,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,DTW
3,100,2016-01-13,45,12478,1247803,31703,JFK,"New York, NY",NY,36,New York,SEA
4,1000,2016-01-17,80,15376,1537602,30436,TUS,"Tucson, AZ",AZ,4,Arizona,DFW
5,10000,2016-01-14,271,12892,1289203,32575,LAX,"Los Angeles, CA",CA,6,California,MIA
6,100000,2016-01-18,2035,15024,1502403,34945,STT,"Charlotte Amalie, VI",VI,78,U.S. Virgin Islands,SJU
7,100001,2016-01-18,2036,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,BOS
8,100002,2016-01-18,2051,10721,1072102,30721,BOS,"Boston, MA",MA,25,Massachusetts,MCO
9,100003,2016-01-18,2054,14843,1484304,34819,SJU,"San Juan, PR",PR,72,Puerto Rico,FLL
